In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from getpass import getpass
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import Testset
from ragas.llms import LangchainLLMWrapper
from docu_bot.utils import create_chatopenai_model, create_openai_embeddings
from docu_bot.document_loaders.git_document_loader import GitDocumentLoader
from docu_bot.document_loaders.utils import LoadedRepositoriesAndFiles
from docu_bot.datasets.generate_synthetic_data_ragas import (
    generate_dataset,
    create_generator,
)
from docu_bot.evaluation.evaluate import Evaluator
from docu_bot.retrievals.document_retrival import DocumentRetrieval
from docu_bot.stores.docstore import DocumentStore
from docu_bot.stores.utils import create_vector_store_from_document_loader, LoadedVectorStores

c:\Users\qwert\Git\Docu-Bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_type = "gpt-4o-mini"
embedding_model_type="text-embedding-3-small"
api_key = getpass("Enter your OpenAI API key: ")

## Generate Sythetic Datase


In [ ]:
llm_model = LangchainLLMWrapper(create_chatopenai_model(model_type=model_type, api_key=api_key))
embeddings_model = LangchainEmbeddingsWrapper(create_openai_embeddings(model_type=embedding_model_type, api_key=api_key))
generator = create_generator(llm_model, embeddings_model)
document_loader = GitDocumentLoader(
    repo_path="https://github.com/jinymusim/serve-model.git", branch="main", loaded_repositories_and_files=LoadedRepositoriesAndFiles()
)

In [5]:
synthetic_data = generate_dataset(generator, document_loader.load(), dataset_size=50)

Generating Samples: 100%|██████████| 40/40 [00:07<00:00,  5.23it/s]


In [ ]:
synthetic_data.to_jsonl(
    os.path.join(os.path.abspath(''), "..", "datasets", "synthetic_data.jsonl"),
)

### Use Sythetic Data to test the model

In [3]:
model_type = "Llama-3.2-3B"
api_key = getpass("Enter your Metacentrum API key: ")
evaluator_llm_model_type = "gpt-4o-mini"
evaluator_embeddings_model_type = "text-embedding-3-small"
open_ai_api_key = getpass("Enter your OpenAI API key: ")

In [5]:
synthetic_data = Testset.from_jsonl(os.path.join(os.path.abspath(''), "..", "datasets", "synthetic_data.jsonl"))
document_loader = GitDocumentLoader(
    repo_path="https://github.com/jinymusim/serve-model.git", branch="main", loaded_repositories_and_files=LoadedRepositoriesAndFiles()
)
docstore =DocumentStore()
cached_vector_store = LoadedVectorStores()
vector_store =create_vector_store_from_document_loader(document_loader, docstore, cached_vector_store)

In [7]:
evaluator = Evaluator(
    evaluator_llm=create_chatopenai_model(model_type=evaluator_llm_model_type, api_key=open_ai_api_key),
    evaluator_embedding_model=create_openai_embeddings(model_type=evaluator_embeddings_model_type, api_key=open_ai_api_key),
)

In [11]:
eval_results = evaluator.evaluate_configuration(
    dataset=synthetic_data,
    rag_llm=create_chatopenai_model(model_type=model_type, api_key=api_key),
    document_retriever=DocumentRetrieval(
        vectorstore=vector_store, 
        docstore=docstore, 
        search_kwargs={"min_score": 0.5, "k": 1})
)

Evaluating: 100%|██████████| 320/320 [02:50<00:00,  1.88it/s]


In [13]:
eval_results

{'factual_correctness': 0.1630, 'faithfulness': 0.2556, 'context_recall': 0.0375, 'semantic_similarity': 0.5644, 'non_llm_context_recall': 0.0000, 'llm_context_precision_with_reference': 0.0500, 'non_llm_context_precision_with_reference': 0.0000, 'context_entity_recall': 0.0224}